<h1>Ranking

<h4>Import packages required

In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import pylab
from collections import Counter
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
import pickle
import networkx as nx
import operator
import math

<p> Fit a TfidfVectorizer and save it

In [301]:
merged_train = pd.read_csv('../data/merged_data/merged_train_textClean.csv', sep=',', header=0)
Tfidf = TfidfVectorizer()
Tfidf.fit(merged_train.body.tolist())
pickle.dump(Tfidf, open("../data/merged_data/Tfidfvectorizer.pickle", "wb"))

In [203]:
def get_emails_ids_per_sender(training):
    sender_list = list(set(training.sender.tolist()))
    emails_ids_per_sender = {}    
    for s in sender_list:
        emails_ids_per_sender[s] = [int(x) for x in training[training.sender==s]['mid'].tolist()]
    return emails_ids_per_sender

In [250]:
def rcy(t_new,t_m):
    t_new = np.datetime64(t_new).astype('datetime64[D]')
    t_m = np.datetime64(t_m).astype('datetime64[D]')
    return round(((t_new - t_m).astype(float))**(-0.85),10)

def cnt(c_new,c_m,Tfidf=TfidfVectorizer()):
    c_new = Tfidf.transform([c_new]).toarray()
    c_new = np.nan_to_num(c_new)
    c_m = Tfidf.transform([c_m]).toarray()
    c_m = np.nan_to_num(c_m)
    return round(abs(1 - spatial.distance.cosine(c_new, c_m)),5)

In [287]:
def rk(score_dic):
    return {key: rank for rank, key in enumerate(sorted(score_dic, key=score_dic.get, reverse=True), 1)}

def Rank_all(rank_r, rank_c, alpha=0.6):
    rank = {}
    for k in rank_r.iterkeys():
        r_r = alpha/rank_r[k] if rank_r.get(k,0)!=0 else 0
        r_c = (1-alpha)/rank_c[k] if rank_c.get(k,0)!=0 else 0
        rank[k] = r_r + r_c
    sorted_rank = [x[0] for x in sorted(rank.items(), key=operator.itemgetter(1),reverse=True)]        
    return sorted_rank

In [219]:
def CC_rank(new_mail,address_books,contact_books,df,Tfidf=TfidfVectorizer()): 
    
    mid = new_mail[0]
    sender = new_mail[1]
    t_new = new_mail[2]
    b_new = new_mail[3]
    
    G_recency = nx.DiGraph()
    G_content = nx.DiGraph()
    all_nodes = [sender] + address_books[sender]
    edges = [(i,j) for i in all_nodes for j in all_nodes if all_nodes.index(j) > all_nodes.index(i)]
    for edge in edges:
        rcy_out = 0
        rcy_in = 0
        cnt_out = 0
        cnt_in = 0
        if contact_books.has_key(edge[0]) and contact_books[edge[0]].has_key(edge[1]):
            dates = df[df.mid.isin(contact_books[edge[0]][edge[1]])]['date'].tolist()
            for d in dates:
                r_out = rcy(t_new,d)
                rcy_out += 1 if math.isnan(r_out) or math.isinf(r_out) else r_out

            bodies = df[df.mid.isin(contact_books[edge[0]][edge[1]])]['body'].tolist()
            for b in bodies:
                c_out = cnt(b_new,b,Tfidf=tfidf)
                cnt_out += 1 if math.isnan(c_out) or math.isinf(c_out) else c_out

        if contact_books.has_key(edge[1]) and contact_books[edge[1]].has_key(edge[0]):
            dates = df[df.mid.isin(contact_books[edge[1]][edge[0]])]['date'].tolist()
            for d in dates:
                r_in = rcy(t_new,d)
                rcy_in += 1 if math.isnan(r_in) or math.isinf(r_in) else r_in

            bodies = df[df.mid.isin(contact_books[edge[1]][edge[0]])]['body'].tolist()
            for b in bodies:
                c_in = cnt(b_new,b,Tfidf=tfidf)
                cnt_in += 1 if math.isnan(c_in) or math.isinf(c_in) else c_in

        
        
        if rcy_out!=0:
            G_recency.add_edge(edge[0],edge[1],weight=(6*rcy_out + rcy_in))
            G_content.add_edge(edge[0],edge[1],weight=(6*cnt_out + cnt_in))
            
        if rcy_in!=0:
            G_recency.add_edge(edge[1],edge[0],weight=(6*rcy_in + rcy_out))
            G_content.add_edge(edge[1],edge[0],weight=(6*cnt_in + cnt_out))
            
            
#         if (6*rcy_out + rcy_in)>0:
#             G_recency.add_edge(edge[0],edge[1],weight=(6*rcy_out + rcy_in))
#             G_recency.add_edge(edge[1],edge[0],weight=(6*rcy_in + rcy_out))
#             if (6*cnt_out + cnt_in)>0:
#                 G_content.add_edge(edge[0],edge[1],weight=(6*cnt_out + cnt_in))
#                 G_content.add_edge(edge[1],edge[0],weight=(6*cnt_in + cnt_out))
#             if (6*cnt_out + cnt_in)<=0:
#                 G_content.add_edge(edge[0],edge[1],weight=0)
#                 G_content.add_edge(edge[1],edge[0],weight=0)


#         if (6*rcy_out + rcy_in)<0:
#             G_recency.add_edge(edge[0],edge[1],weight=0)
#             G_recency.add_edge(edge[1],edge[0],weight=0)
            
#         if (6*cnt_out + cnt_in)>0:
#             G_content.add_edge(edge[0],edge[1],weight=(6*cnt_out + cnt_in))
#             G_content.add_edge(edge[1],edge[0],weight=(6*cnt_in + cnt_out))
#         if (6*cnt_out + cnt_in)<0:
#             G_content.add_edge(edge[0],edge[1],weight=0)
#             G_content.add_edge(edge[1],edge[0],weight=0)
    
#     nx.draw(G_recency)
#     plt.show()
    
    rank_r = nx.algorithms.centrality.closeness_centrality(G_recency,distance='weight',normalized=True)
    rank_r = rk(rank_r)
    rank_c = nx.algorithms.centrality.closeness_centrality(G_content,distance='weight',normalized=True)
    rank_c = rk(rank_c)
    Rank = Rank_all(rank_r,rank_c)
    if sender in Rank:
        Rank.remove(sender)
    return Rank[:10]

In [282]:
def Rcy_rank(new_mail,address_books,contact_books,df):
    mid = new_mail[0]
    sender = new_mail[1]
    t_new = new_mail[2]
    b_new = new_mail[3]
    
    ran = {}

    for rec in address_books[sender]:
        score = 0
        cache = df[df.mid.isin(contact_books[sender][rec])]['date'].tolist()
        for d in cache:
            r = rcy(t_new,d)
            score += 1 if math.isnan(r) or math.isinf(r) else r
            
        ran[rec] = score
#     sorted_rank = [x[0] for x in sorted(ran.items(), key=operator.itemgetter(1),reverse=True)]
#     return sorted_rank[:10]
    return rk(ran)

In [299]:
def Cnt_rank(new_mail,address_books,contact_books,df,Tfidf=TfidfVectorizer()):
    mid = new_mail[0]
    sender = new_mail[1]
    t_new = new_mail[2]
    b_new = Tfidf.transform([new_mail[3]]).toarray()
#     print b_new.shape
    ran = {}
    
    for rec in address_books[sender]:
        score = 0
        cache = df[df.mid.isin(contact_books[sender][rec])]['body'].tolist()
        a_m = Tfidf.transform(cache).toarray()
        for row in a_m:
#             print row.shape
            score += abs(spatial.distance.cosine(b_new, row))

        ran[rec] = score
    return rk(ran)

In [292]:
tfidf = pickle.load(open("../data/merged_data/Tfidfvectorizer.pickle", "rb"))

test = pd.read_csv('../data/merged_data/eva_test.csv', sep=',', header=0)
train = pd.read_csv('../data/merged_data/eva_train.csv', sep=',', header=0)

emails_ids_per_sender = get_emails_ids_per_sender(train)
all_senders = emails_ids_per_sender.keys()

address_books = {}
for sender in all_senders:
    book = train[train.sender==sender]['recipients'].tolist()
    book = [x for subitem in book for x in subitem.split(" ")]
    book = list(set(book))
    address_books[sender] = book

contact_books = {}
for sender in all_senders:
    mails = train[train.sender==sender]
    book = {}
    for rec in address_books[sender]:
        rec_mail =mails[mails.recipients.str.contains(rec)]['mid'].tolist()
        book[rec] = rec_mail
    contact_books[sender] = book

In [220]:
with open("../results/networkx_result_v0.1.txt", 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    i = 0
    for index,serie in test.iterrows():
        row = serie.tolist()
        mid = str(row[0])
        rank = CC_rank(row,address_books,contact_books,train,Tfidf=tfidf)
        my_file.write(mid + ',' + ' '.join(rank) + '\n')

        i += 1
        if i%50==0:
            print i
        
    my_file.close()
    

/anaconda/envs/Python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in double_scalars


In [300]:
with open("../results/Cnt_result_v0.1.txt", 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    i = 0
    for index,serie in test.iterrows():
        row = serie.tolist()
        mid = str(row[0])
#         d1 = Rcy_rank(row,address_books,contact_books,train)
        d2 = Cnt_rank(row,address_books,contact_books,train,Tfidf=tfidf)
#         rn = Rank_all(d1,d2)[:10]
        rn = [x[0] for x in sorted(d2.items(), key=operator.itemgetter(1),reverse=True)]
        my_file.write(mid + ',' + ' '.join(rn[:10]) + '\n')

        i += 1
        if i%50==0:
            print i
        
    my_file.close()